In [1]:
import numpy
import numpy.random
import pandas
import vtreat

In [2]:
numpy.random.seed(2019)

In [3]:
n_rows = 1000
y_levels = ['a', 'b', 'c']
d = pandas.DataFrame({'y': numpy.random.choice(y_levels, size=n_rows)})
# signal variables, correlated with y-levels
for i in range(2):
    vmap = {yl: numpy.random.normal(size=1)[0] for yl in y_levels}
    d['var_n_' + str(i)] = [vmap[li] + numpy.random.normal(size=1)[0] for li in d['y']]
for i in range(2):
    col = numpy.random.choice(y_levels, size=n_rows)
    col = [col[i] if numpy.random.uniform(size=1)[0]<=0.8 else d['y'][i] for i in range(n_rows)]
    d['var_c_' + str(i)] = col
# noise variables, uncorrelated with y-levels
for i in range(2):
    d['noise_n_' + str(i)] = [numpy.random.normal(size=1)[0] + numpy.random.normal(size=1)[0] for li in d['y']]
for i in range(2):
    d['noise_c_' + str(i)] = numpy.random.choice(y_levels, size=n_rows)
d.head()

,y,var_n_0,var_n_1,var_c_0,var_c_1,noise_n_0,noise_n_1,noise_c_0,noise_c_1
0,a,0.544891,1.595448,b,a,1.196747,-0.714955,a,a
1,c,-0.433273,0.778452,a,b,0.332786,-1.175452,a,b
2,b,-1.230834,0.859162,b,a,-1.454101,-3.079244,a,b
3,a,1.158161,-0.344363,c,b,-1.175055,-1.314302,a,b
4,c,-1.029690,0.789506,c,c,0.809634,0.951092,b,c


In [4]:
treatment = vtreat.MultinomialOutcomeTreatment(
    outcome_name='y')
cross_frame = treatment.fit_transform(d, d['y'])
score_frame = treatment.score_frame_

To select variables we either make our selection in terms of new variables as follows.

In [5]:
good_new_variables = score_frame.variable[score_frame.recommended].unique()
good_new_variables

array(['var_n_0', 'var_n_1', 'var_c_0_logit_code_a',
       'var_c_0_prevalence_code', 'var_c_0_lev_b', 'var_c_0_lev_a',
       'var_c_0_lev_c', 'var_c_1_logit_code_a', 'var_c_1_prevalence_code',
       'var_c_1_lev_a', 'var_c_0_logit_code_b', 'var_c_1_logit_code_b',
       'var_c_1_lev_b', 'var_c_1_lev_c', 'var_c_0_logit_code_c',
       'var_c_1_logit_code_c'], dtype=object)

Or in terms of original variables as follows.

In [6]:
good_original_variables = score_frame.orig_variable[score_frame.recommended].unique()
good_original_variables

array(['var_n_0', 'var_n_1', 'var_c_0', 'var_c_1'], dtype=object)

Notice, in each case we must call unique as each variable (derived or original) is potentially qualified against each possible outcome.

The cross frame and score frame look like the following.

In [7]:
cross_frame.head()

,y,var_n_0,var_n_1,var_c_0_logit_code_a,var_c_0_logit_code_c,var_c_0_logit_code_b,var_c_0_prevalence_code,var_c_0_lev_b,var_c_0_lev_a,var_c_0_lev_c,var_c_1_logit_code_a,var_c_1_logit_code_c,var_c_1_logit_code_b,var_c_1_prevalence_code,var_c_1_lev_a,var_c_1_lev_b,var_c_1_lev_c
0,a,0.544891,1.595448,-0.183425,-0.246066,0.333199,0.352,1.0,0.0,0.0,0.164827,-0.258790,0.046310,0.346,1.0,0.0,0.0
1,c,-0.433273,0.778452,0.320872,-0.144157,-0.254121,0.328,0.0,1.0,0.0,-0.149856,-0.257395,0.322305,0.334,0.0,1.0,0.0
2,b,-1.230834,0.859162,-0.233913,-0.186684,0.326693,0.352,1.0,0.0,0.0,0.209085,-0.287449,0.023177,0.346,1.0,0.0,0.0
3,a,1.158161,-0.344363,-0.220148,0.290051,-0.199207,0.320,0.0,0.0,1.0,-0.149856,-0.257395,0.322305,0.334,0.0,1.0,0.0
4,c,-1.029690,0.789506,-0.193397,0.283625,-0.212830,0.320,0.0,0.0,1.0,-0.117681,0.397098,-0.522505,0.320,0.0,0.0,1.0


In [8]:
treatment.score_frame_

,variable,orig_variable,treatment,y_aware,has_range,PearsonR,significance,vcount,default_threshold,recommended,outcome_target
0,var_n_0,var_n_0,clean_copy,False,True,0.460713,1.079644e-53,4.0,0.062500,True,a
1,var_n_1,var_n_1,clean_copy,False,True,-0.253237,4.259663e-16,4.0,0.062500,True,a
2,noise_n_0,noise_n_0,clean_copy,False,True,0.003113,9.216881e-01,4.0,0.062500,False,a
3,noise_n_1,noise_n_1,clean_copy,False,True,-0.030264,3.390479e-01,4.0,0.062500,False,a
4,var_c_0_logit_code_a,var_c_0,logit_code,True,True,0.180222,9.520214e-09,12.0,0.020833,True,a
...,...,...,...,...,...,...,...,...,...,...,...
91,noise_c_1_logit_code_b,noise_c_1,logit_code,True,True,0.015519,6.240063e-01,12.0,0.020833,False,c
92,noise_c_1_prevalence_code,noise_c_1,prevalence_code,False,True,-0.028448,3.688353e-01,4.0,0.062500,False,c
93,noise_c_1_lev_c,noise_c_1,indicator_code,False,True,-0.012951,6.825051e-01,12.0,0.020833,False,c
94,noise_c_1_lev_b,noise_c_1,indicator_code,False,True,-0.036427,2.497826e-01,12.0,0.020833,False,c


## Conclusion

In all cases (classification, regression, unsupervised, and multinomial classification) the intent is that `vtreat` transforms are essentially one liners.

The preparation commands are organized as follows:

 * **Regression**: [`R` regression example](https://github.com/WinVector/vtreat/blob/master/Examples/Regression/Regression.md), [`Python` regression example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Regression/Regression.md).
 * **Classification**: [`R` classification example](https://github.com/WinVector/vtreat/blob/master/Examples/Classification/Classification.md), [`Python` classification  example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Classification/Classification.md).
 * **Unsupervised tasks**: [`R` unsupervised example](https://github.com/WinVector/vtreat/blob/master/Examples/Unsupervised/Unsupervised.md), [`Python` unsupervised example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Unsupervised/Unsupervised.md).
 * **Multinomial classification**: [`R` multinomial classification example](https://winvector.github.io/vtreat/articles/MultiClassVtreat.html), [`Python` multinomial classification example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/MultinomialExample.ipynb).

These current revisions of the examples are designed to be small, yet complete.  So as a set they have some overlap, but the user can rely mostly on a single example for a single task type.
